In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle
import joblib

# Summary 
Necessary steps to upload model to Vertex AI for implementation

## Input 
Some basic input about location, storage,... 

In [2]:
MODEL_ARTIFACT_DIR = "model"  # @param {type:"string"}
REPOSITORY = "experiment-iris-repository"  # @param {type:"string"}
IMAGE = "experiment_iris_image"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "experiment_iris_model"  # @param {type:"string"}
USER_SRC_DIR = "src"  # @param {type:"string"}
LOCAL_MODEL_ARTIFACTS_DIR = "model"  # @param {type:"string"}
REGION='asia-southeast1'
PROJECT_ID='teak-store-392915'
BUCKET_URI='gs://gcp_asia/ml_pipeline_experiment_20240825'

Create requirements.txt which contains all necessary packages 

In [5]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

Writing requirements.txt


Create and load preprocessing module. Its to preprocess the data before predict with model   
Iris dataset is simple so I decide do nothing so keep it blank. 

In [8]:
%%writefile $USER_SRC_DIR/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
#         if self._means is None:  # during training only
#             self._means = np.mean(data, axis=0)

#         if self._stds is None:  # during training only
#             self._stds = np.std(data, axis=0)
#             if not self._stds.all():
#                 raise ValueError("At least one column has standard deviation of 0.")

        # return (data - self._means) / self._stds
         return data

Writing src/preprocess.py


In [29]:
%cd $USER_SRC_DIR/

import pickle

import joblib
from preprocess import MySimpleScaler

scaler = MySimpleScaler()

with open(f"../{LOCAL_MODEL_ARTIFACTS_DIR}/preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

/home/jupyter/ml_pipeline/experiment_iris/src


In [ ]:
# %cd ..
# !gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
# !gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Create predictor class to control following steps:
- Preprocessor
- Predictor
- Handler 

In [20]:
%cd /home/jupyter/ml_pipeline/experiment_iris

/home/jupyter/ml_pipeline/experiment_iris


In [12]:
%%writefile predictor.py

import joblib
import numpy as np
import pickle

from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils

from sklearn.datasets import load_iris


class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        prediction_utils.download_model_artifacts(artifacts_uri)

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._model = joblib.load("decision_tree_model_iris_v1.joblib")
        self._preprocessor = preprocessor

    def predict(self, instances):
        """Performs prediction."""
        inputs = np.asarray(instances)
        preprocessed_inputs = self._preprocessor.preprocess(inputs)
        outputs = self._model.predict(preprocessed_inputs)

        return {"predictions": [self._class_names[class_num] for class_num in outputs]}
    

Overwriting predictor.py


## Package model into a container object 

In [101]:
import os
import docker
from google.cloud.aiplatform.prediction import LocalModel
from src.predictor import  CprPredictor  # Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    base_image="python:3.7",
    predictor=CprPredictor,  # Update this to the custom predictor class.
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

In [102]:
local_model.get_serving_container_spec()

image_uri: "asia-southeast1-docker.pkg.dev/teak-store-392915/experiment-iris-repository/experiment_iris_image"
predict_route: "/predict"
health_route: "/health"

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

If `artifactregistry.googleapis.com` is not enabled in your project, enable the API before proceeding.

In [5]:
!gcloud services list

NAME                                         TITLE
aiplatform.googleapis.com                    Vertex AI API
analyticshub.googleapis.com                  Analytics Hub API
apigateway.googleapis.com                    API Gateway API
appengine.googleapis.com                     App Engine Admin API
artifactregistry.googleapis.com              Artifact Registry API
automl.googleapis.com                        Cloud AutoML API
autoscaling.googleapis.com                   Cloud Autoscaling API
backupdr.googleapis.com                      Backup and DR Service API
bigquery.googleapis.com                      BigQuery API
bigqueryconnection.googleapis.com            BigQuery Connection API
bigquerydatapolicy.googleapis.com            BigQuery Data Policy API
bigquerydatatransfer.googleapis.com          BigQuery Data Transfer API
bigquerymigration.googleapis.com             BigQuery Migration API
bigqueryreservation.googleapis.com           BigQuery Reservation API
bigquerystorage.googleapis

If `artifactregistry.googleapis.com` is not enabled in your project, enable the API before proceeding.

In [ ]:
#!gcloud services enable artifactregistry.googleapis.com

In [10]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

Create request issued for: [experiment-iris-repository]
Waiting for operation [projects/teak-store-392915/locations/asia-southeast1/ope
rations/77ff87c9-19d0-4619-9999-2c6b0ae6b740] to complete...done.              
Created repository [experiment-iris-repository].


In [11]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Adding credentials for: asia-southeast1-docker.pkg.dev
Docker configuration file updated.


Use SDK to push the image.

In [103]:
local_model.push_image()

## Deploy to Vertex AI

In [6]:
from google.cloud import aiplatform

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Use the LocalModel instance to upload the model. It will populate the container spec automatically for you.

In [104]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

Creating Model
Create Model backing LRO: projects/958509004100/locations/asia-southeast1/models/8350861181702897664/operations/6303553488353755136
Model created. Resource name: projects/958509004100/locations/asia-southeast1/models/8350861181702897664@1
To use this Model in another session:
model = aiplatform.Model('projects/958509004100/locations/asia-southeast1/models/8350861181702897664@1')
